🚗 **Dynamic Parking Pricing System - Final Notebook**

In [ ]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

In [ ]:
# 📦 Imports
import pandas as pd
import numpy as np
import pathway as pw
import datetime
import bokeh.plotting
from bokeh.io import output_file, save
from google.colab import files
import json

In [ ]:
# 📊 Step 1: Load and preprocess data
df = pd.read_csv("/content/dataset.csv")
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')
df = df.sort_values('Timestamp').reset_index(drop=True)
df = df.head(300)  # LIMIT TO 300 ROWS FOR SPEED
df[['Timestamp', 'Occupancy', 'Capacity']].to_csv("parking_stream.csv", index=False)

In [ ]:
# 🧱 Step 2: Define schema
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int

In [ ]:
# ⏱️ Step 3: Read CSV in STATIC mode
source = pw.io.csv.read("parking_stream.csv", schema=ParkingSchema, mode="static")

In [ ]:
# 🕒 Step 4: Format timestamp
date_format = "%Y-%m-%d %H:%M:%S"
data_with_time = source.with_columns(
    t=source.Timestamp.dt.strptime(date_format),
    day=source.Timestamp.dt.strptime(date_format).dt.strftime("%Y-%m-%dT00:00:00")
)

In [ ]:
# 🧠 Step 5: Define demand function and models
vehicle_weights = {"car": 1.0, "bike": 0.5, "truck": 1.5, "cycle": 0.3}
traffic_weights = {"low": 0.5, "average": 1.0, "high": 1.5}

def compute_demand(occ, cap):
    occ_rate = occ / cap if cap != 0 else 0
    return (
        1.2 * occ_rate +
        0.8 * 3 -
        0.5 * traffic_weights["low"] +
        0.9 * 0 +
        0.7 * vehicle_weights["car"]
    )

def compute_price(demand):
    norm_demand = demand / 10
    return max(5, min(20, 10 * (1 + 0.5 * norm_demand)))

def build_model_1(data):
    return (
        data.windowby(
            pw.this.t,
            instance=pw.this.day,
            window=pw.temporal.tumbling(datetime.timedelta(days=1)),
            behavior=pw.temporal.exactly_once_behavior()
        )
        .reduce(
            t=pw.this._pw_window_end,
            occ_max=pw.reducers.max(pw.this.Occupancy),
            cap=pw.reducers.max(pw.this.Capacity)
        )
        .with_columns(
            unclamped_price=(10 + 5 * (pw.this.occ_max / pw.this.cap))
        )
        .with_columns(
            price=pw.apply(lambda p: max(5, min(20, p)), pw.this.unclamped_price)
        )
    )

def build_model_2(data):
    return (
        data.windowby(
            pw.this.t,
            instance=pw.this.day,
            window=pw.temporal.tumbling(datetime.timedelta(days=1)),
            behavior=pw.temporal.exactly_once_behavior()
        )
        .reduce(
            t=pw.this._pw_window_end,
            occ_max=pw.reducers.max(pw.this.Occupancy),
            cap=pw.reducers.max(pw.this.Capacity)
        )
        .with_columns(
            demand=pw.apply(compute_demand, pw.this.occ_max, pw.this.cap)
        )
        .with_columns(
            price=pw.apply(compute_price, pw.this.demand)
        )
    )


def build_model_3(data):
    return (
        data.windowby(
            pw.this.t,
            instance=pw.this.day,
            window=pw.temporal.tumbling(datetime.timedelta(days=1)),
            behavior=pw.temporal.exactly_once_behavior()
        )
        .reduce(
            t=pw.this._pw_window_end,
            occ_max=pw.reducers.max(pw.this.Occupancy),
            cap=pw.reducers.max(pw.this.Capacity)
        )
        .with_columns(
            price=pw.this.occ_max / pw.this.cap * 10
        )
    )

In [ ]:
# 🚀 Step 6: Run and export all models
model1 = build_model_1(data_with_time)
model2 = build_model_2(data_with_time)
model3 = build_model_3(data_with_time)

writer1 = pw.io.jsonlines.write(model1, "model1.jsonl")
writer2 = pw.io.jsonlines.write(model2, "model2.jsonl")
writer3 = pw.io.jsonlines.write(model3, "model3.jsonl")

pw.run()
writer1.wait()
writer2.wait()
writer3.wait()

In [ ]:
# 📈 Step 7: Load and visualize models
model_dfs = {}
for i in [1, 2, 3]:
    with open(f"model{i}.jsonl") as f:
        lines = [json.loads(line) for line in f]
    df = pd.DataFrame(lines)
    timestamp_col = [col for col in df.columns if col.startswith("t")][0]
    df[timestamp_col] = pd.to_datetime(df[timestamp_col])
    df = df.sort_values(timestamp_col)
    model_dfs[i] = (df, timestamp_col)

output_file("bokeh_output.html")
fig = bokeh.plotting.figure(
    height=400,
    width=800,
    title="Compare Parking Prices: Model 1 (Green), 2 (Blue), 3 (Red)",
    x_axis_type="datetime"
)
colors = {1: "green", 2: "blue", 3: "red"}
for i in [1, 2, 3]:
    df, t_col = model_dfs[i]
    fig.line(df[t_col], df["price"], line_width=2, color=colors[i], legend_label=f"Model {i}")
    fig.circle(df[t_col], df["price"], size=5, color=colors[i])

fig.legend.location = "top_left"
save(fig)

In [ ]:
# 💾 Step 8: Download the final chart
files.download("bokeh_output.html")